# Working with Models in AutoGen

This lesson explores how to work with different model clients in AutoGen, focusing on OpenAI and Azure OpenAI integration. We'll learn how to set up and use these clients for AI model interactions.

## Setup

In [1]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# Set up necessary environment variables
_set_env("OPENAI_API_KEY")
# _set_env("AZURE_API_KEY")

In [3]:
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("Async environment configured for Jupyter.")
except ImportError:
    print("Please install nest_asyncio with `pip install nest_asyncio`")

Async environment configured for Jupyter.


## OpenAI Integration

Let's start by exploring how to use OpenAI models with AutoGen. First, we'll need to install the required package:

In [15]:
# Install the OpenAI extension (run this in a terminal)
# pip install "autogen-ext[openai]"
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import UserMessage

# Create an OpenAI client
openai_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
    temperature=0.7,  #  Adding temperature for more creative responses
)

In [17]:
await openai_client.create([UserMessage(content="Hello, how are you?", source="user")])

CreateResult(finish_reason='stop', content="Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", usage=RequestUsage(prompt_tokens=14, completion_tokens=30), cached=False, logprobs=None)

We can also get structured output!

In [11]:
# Install the OpenAI extension (run this in a terminal)
# pip install "autogen-ext[openai]"
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import UserMessage

# Create an OpenAI client
openai_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
    temperature=0.7,  #  Adding temperature for more creative responses
    response_format={"type": "json_object"}
)

In [13]:
response = await openai_client.create([UserMessage(content="Create a database as a json object with 10 profiles for customers containing name, email, purchasing preferences.", source="user")])
response.content

'{"customers": [\n    {\n        "name": "Alice Smith",\n        "email": "alice.smith@example.com",\n        "purchasing_preferences": {\n            "categories": ["electronics", "books"],\n            "frequency": "monthly",\n            "average_spend": 150\n        }\n    },\n    {\n        "name": "Bob Johnson",\n        "email": "bob.johnson@example.com",\n        "purchasing_preferences": {\n            "categories": ["clothing", "accessories"],\n            "frequency": "bi-weekly",\n            "average_spend": 80\n        }\n    },\n    {\n        "name": "Charlie Brown",\n        "email": "charlie.brown@example.com",\n        "purchasing_preferences": {\n            "categories": ["home & garden", "furniture"],\n            "frequency": "quarterly",\n            "average_spend": 300\n        }\n    },\n    {\n        "name": "Diana Prince",\n        "email": "diana.prince@example.com",\n        "purchasing_preferences": {\n            "categories": ["health & beauty", "fitn

In [14]:
import json

json.loads(response.content)

{'customers': [{'name': 'Alice Smith',
   'email': 'alice.smith@example.com',
   'purchasing_preferences': {'categories': ['electronics', 'books'],
    'frequency': 'monthly',
    'average_spend': 150}},
  {'name': 'Bob Johnson',
   'email': 'bob.johnson@example.com',
   'purchasing_preferences': {'categories': ['clothing', 'accessories'],
    'frequency': 'bi-weekly',
    'average_spend': 80}},
  {'name': 'Charlie Brown',
   'email': 'charlie.brown@example.com',
   'purchasing_preferences': {'categories': ['home & garden', 'furniture'],
    'frequency': 'quarterly',
    'average_spend': 300}},
  {'name': 'Diana Prince',
   'email': 'diana.prince@example.com',
   'purchasing_preferences': {'categories': ['health & beauty', 'fitness'],
    'frequency': 'monthly',
    'average_spend': 100}},
  {'name': 'Ethan Hunt',
   'email': 'ethan.hunt@example.com',
   'purchasing_preferences': {'categories': ['travel', 'experiences'],
    'frequency': 'annually',
    'average_spend': 2000}},
  {'nam

Running the basic model client from an async function.

In [21]:
# Example: Create a weather report assistant
async def generate_weather_report():
    prompt = "Create a weather report for New York City with made-up but realistic-sounding data."
    messages = [UserMessage(content=prompt, source="user")]
    
    result = await openai_client.create(messages)
    return result.content    

# Run the weather report generator

response = await generate_weather_report()

In [22]:
response

'**New York City Weather Report**  \n**Date: October 13, 2023**  \n\n**Current Conditions:**  \n- Temperature: 68°F (20°C)  \n- Humidity: 62%  \n- Wind: NW at 12 mph  \n- Sky Condition: Partly cloudy  \n\n**Morning Forecast:**  \nExpect a mild start to the day with temperatures around 65°F (18°C). Winds will be light, shifting from the northwest. A few clouds may linger, but overall, conditions will be pleasant for morning commuters.\n\n**Afternoon Forecast:**  \nTemperatures will rise to a comfortable high of 73°F (23°C). Expect partly sunny skies with occasional breaks of sunshine. Winds will increase slightly, reaching up to 15 mph, providing a nice breeze. Ideal weather for outdoor activities!\n\n**Evening Forecast:**  \nAs the sun sets, temperatures will drop to around 60°F (16°C). Skies will remain partly cloudy, and a light wind will continue. A perfect evening for a stroll in the park or dining al fresco.\n\n**Extended Outlook:**  \n- **Saturday:** A high of 75°F (24°C) with mo

## Azure OpenAI Integration

Now let's look at how to use Azure OpenAI services:

In [ ]:
# Install Azure extensions (run this in a terminal)
# pip install "autogen-ext[openai,azure]"

from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Set up Azure OpenAI client
def setup_azure_client():
    # Create token provider for authentication
    token_provider = get_bearer_token_provider(
        DefaultAzureCredential(), 
        "https://cognitiveservices.azure.com/.default"
    )
    
    return AzureOpenAIChatCompletionClient(
        azure_deployment="your-deployment-name",
        model="gpt-4",
        api_version="2024-02-15-preview",
        azure_endpoint="https://your-resource.openai.azure.com/",
        azure_ad_token_provider=token_provider
    )

# Example: Create a recipe generator using Azure OpenAI
async def generate_recipe():
    azure_client = setup_azure_client()
    
    prompt = "Create a unique recipe for a healthy breakfast smoothie."
    messages = [UserMessage(content=prompt, source="user")]
    
    result = await azure_client.create(messages)
    print("Recipe:", result.content)

## Working with Multiple Models

Here's an example of how to switch between different models based on the task:

In [25]:
from typing import Union
from dataclasses import dataclass

@dataclass
class ModelConfig:
    client: Union[OpenAIChatCompletionClient]
    max_tokens: int
    temperature: float

async def smart_model_selector(task_type: str, content: str):
    # Configure different models for different tasks
    models = {
        "creative": ModelConfig(
            client=OpenAIChatCompletionClient(
                model="gpt-4o-mini",
                temperature=0.9
            ),
            max_tokens=500,
            temperature=0.9
        ),
        "analytical": ModelConfig(
            client=OpenAIChatCompletionClient(
                model="gpt-4o",
                temperature=0.1
            ),
            max_tokens=300,
            temperature=0.1
        )
    }
    
    # Select appropriate model based on task
    model_config = models.get(task_type, models["analytical"])
    
    # Create and process the message
    messages = [UserMessage(content=content, source="user")]
    result = await model_config.client.create(messages)
    
    return result.content

# Example usage
async def main():
    # Creative task
    story = await smart_model_selector(
        "creative",
        "Write a short story about a robot learning to paint"
    )
    print("Creative Story:", story)
    
    # Analytical task
    analysis = await smart_model_selector(
        "analytical",
        "Analyze the trends in renewable energy adoption"
    )
    print("Analysis:", analysis)

# Run the example
response = await main()
response

Creative Story: In a small, dimly lit workshop on the outskirts of a bustling city, an old painter named Eli was struggling to find his muse. The once vibrant canvases now lay forgotten, collecting dust, while the vibrant splashes of color he once commanded faded into memory. One rainy afternoon, as thunder rumbled in the distance, a package arrived at his door—a new project he hadn’t anticipated: an AI robot named Aria, designed to assist in artistic pursuits.

Eli's initial reaction was one of skepticism. “How can a robot understand art?” he grumbled, eyeing Aria with a mix of curiosity and disdain. The robot stood silently, its smooth metallic surface reflecting the sparse light of the workshop. With articulated arms and a built-in canvas, Aria was programmed to learn by observation and mimicry.

Reluctantly, Eli began to teach Aria. He set up a canvas, splattering it with vibrant colors and expressive strokes. “Feel the paint,” he said, as if he were talking to a reluctant student.

This lesson covered the basics of working with different model clients in AutoGen. We explored how to:
1. Set up and use OpenAI models
2. Configure Azure OpenAI integration
3. Create a smart model selector for different types of tasks

Remember to always handle your API keys securely and consider rate limits and costs when making API calls to these services.